# Loading NIfTI files and aligning with event data

I'm using VOT distributional learning data as an example here.  The goal is to load in event data to a data frame, and the NIfTI files to another data frame (keyed by subject and run number).  Then split up the behavioral data to create a separate alignment for each run/subject combination.  Finally, concatenate the alignments, NIfTI files, and the event data frames together into a combined dataset.

The goal is to have three things:

1. An event data frame with $n$ rows that describes all the relevant events
2. A big 4d $x,y,z,t$ NIfTI array where $t=1..T$
3. A n×T (sparse) matrix where each row corresponds to an event and each column to a time slice in the NIfTI data.  The entry at (i,t) corresponds to the HRF for event i evaluated at time t.

## Loading behavioral data

In [1]:
using YAML, DataFrames

In [2]:
vot_dir = "/home/dave/work/experiments/adapt-fmri/imaging_analysis"

clean_octals(s) = replace(s, r"subject: 0+", "subject: ")

sessions = joinpath(vot_dir, "session_info.yml") |> readstring |> clean_octals |> YAML.load
sess = sessions[1]

Dict{Any,Any} with 5 entries:
  "image_directory"  => "data-imaging/vot-adapt03/20150430"
  "bold_images"      => String["data-imaging/vot-adapt03/20150430/vot-adapt03_2…
  "anatomical_image" => "data-imaging/vot-adapt03/20150430/vot-adapt03_20150430…
  "behavioral_data"  => "data-behav/vot_adapt_003_scan.csv"
  "subject"          => 3

In [3]:
load_behav(s) = DataFrames.readtable(joinpath(vot_dir, sess["behavioral_data"]))

behav = load_behav(sess)
#behav[:start_secs_relative] = 0.
#by(behav, :run_number) do b
#    b[:, :start_secs_relative] = b[:start_secs] - minimum(b[:start_secs])
#    b
#end

,experiment_name,expt_start_date_time,subject_num,subject_handedness,run_number,scan_or_behav,b_vot_cond,trial_number,category,vot,word,picture_left,picture_right,response,response_keycode,response_key,rt,sound_on_secs,pic_on_secs,start_secs,start_date_time,jitter_trs
1,vot_adapt,30-Apr-2015 13:53:51,3,1,1,scan,-10,1,p,20,BEES20.WAV,bees.png,peas.png,right,31,2@,1.711,0.15,0.006,1992.9611,30-Apr-2015 14:06:37,2
2,vot_adapt,30-Apr-2015 13:53:51,3,1,1,scan,-10,2,b,-10,BEES-10.WAV,peas.png,bees.png,right,31,2@,1.695,0.15,0.002,1998.9611,30-Apr-2015 14:06:42,2
3,vot_adapt,30-Apr-2015 13:53:51,3,1,1,scan,-10,3,p,10,BEACH10.WAV,peach.png,beach.png,left,30,1!,1.095,0.15,0.007,2004.9611,30-Apr-2015 14:06:48,3
4,vot_adapt,30-Apr-2015 13:53:51,3,1,1,scan,-10,4,p,50,BEES50.WAV,peas.png,bees.png,left,30,1!,1.391,0.15,0.009,2013.9611,30-Apr-2015 14:06:54,1
5,vot_adapt,30-Apr-2015 13:53:51,3,1,1,scan,-10,5,p,30,BEACH30.WAV,peach.png,beach.png,left,30,1!,1.311,0.15,0.007,2016.9611,30-Apr-2015 14:07:03,2
6,vot_adapt,30-Apr-2015 13:53:51,3,1,1,scan,-10,6,b,-10,BEACH-10.WAV,peach.png,beach.png,right,31,2@,1.278,0.15,0.003,2022.9611,30-Apr-2015 14:07:06,1
7,vot_adapt,30-Apr-2015 13:53:51,3,1,1,scan,-10,7,p,40,BEAK40.WAV,beak.png,peak.png,right,31,2@,1.927,0.15,0.001,2025.9611,30-Apr-2015 14:07:12,2
8,vot_adapt,30-Apr-2015 13:53:51,3,1,1,scan,-10,8,b,-10,BEACH-10.WAV,beach.png,peach.png,left,30,1!,1.135,0.15,0.006,2031.9611,30-Apr-2015 14:07:15,2
9,vot_adapt,30-Apr-2015 13:53:51,3,1,1,scan,-10,9,b,-10,BEAK-10.WAV,peak.png,beak.png,right,31,2@,1.663,0.15,0.002,2037.9611,30-Apr-2015 14:07:21,1
10,vot_adapt,30-Apr-2015 13:53:51,3,1,1,scan,-10,10,b,-10,BEES-10.WAV,bees.png,peas.png,left,30,1!,1.135,0.15,0.008,2040.9611,30-Apr-2015 14:07:27,2


## Load NIfTI

In [4]:
using NIfTI

function add_prefix(str, prefix)
    path, fn = splitdir(str)
    joinpath(path, prefix * fn)
end

read_one_nifti(fn, prefix; kwargs...) = niread(joinpath(vot_dir, add_prefix(fn, prefix)); kwargs...)

function load_nifti(s; prefix="wra")
    niftis =  read_one_nifti.(s["bold_images"], [prefix])
    return DataFrame(run_number=collect(1:length(niftis)), subject_num=s["subject"], nifti=niftis)
end

load_nifti (generic function with 1 method)

One issue with this particular dataset is that SPM for some reason strips off timing information from where NIfTI.jl expects to find it in the header.

In [5]:
niftis_preproc = read_one_nifti.(sess["bold_images"][1:1], ["", "a", "ra", "wra"], mmap=true)
headers = [ni.header for ni in niftis_preproc]
voxel_size.(headers)
time_step.(headers)

4-element Array{Float64,1}:
 3012.5
    0.0
    0.0
    0.0

Because the dataset itself isn't critical, just use the un-preprocessed NIfTI files.

In [6]:
run_niftis = load_nifti(sess, prefix="")
names(run_niftis)

3-element Array{Symbol,1}:
 :run_number 
 :subject_num
 :nifti      

In [7]:
map(size, run_niftis[:nifti])

4-element DataArrays.DataArray{Any,1}:
 (64,64,30,116)
 (64,64,30,116)
 (64,64,30,116)
 (64,64,30,116)

In [8]:
[time_step(ni.header) for ni in run_niftis[:nifti]]

4-element Array{Float64,1}:
 3012.5
 3012.5
 3012.5
 3012.5

In [9]:
behav_by_run = groupby(behav, :run_number)
behav_by_run[2]

,experiment_name,expt_start_date_time,subject_num,subject_handedness,run_number,scan_or_behav,b_vot_cond,trial_number,category,vot,word,picture_left,picture_right,response,response_keycode,response_key,rt,sound_on_secs,pic_on_secs,start_secs,start_date_time,jitter_trs
1,vot_adapt,30-Apr-2015 13:53:51,3,1,2,scan,-10,1,p,30,BEAK30.WAV,peak.png,beak.png,right,31,2@,1.535,0.15,0.009,2388.0169,30-Apr-2015 14:13:12,1
2,vot_adapt,30-Apr-2015 13:53:51,3,1,2,scan,-10,2,b,0,BEES0.WAV,bees.png,peas.png,left,30,1!,0.863,0.15,0.007,2391.0169,30-Apr-2015 14:13:17,2
3,vot_adapt,30-Apr-2015 13:53:51,3,1,2,scan,-10,3,b,-20,BEAK-20.WAV,peak.png,beak.png,right,31,2@,1.431,0.15,0.003,2397.0169,30-Apr-2015 14:13:20,2
4,vot_adapt,30-Apr-2015 13:53:51,3,1,2,scan,-10,4,p,30,BEAK30.WAV,beak.png,peak.png,left,30,1!,1.007,0.15,0.008,2403.0169,30-Apr-2015 14:13:26,1
5,vot_adapt,30-Apr-2015 13:53:51,3,1,2,scan,-10,5,b,-10,BEES-10.WAV,bees.png,peas.png,left,30,1!,0.967,0.15,0.006,2406.0169,30-Apr-2015 14:13:32,2
6,vot_adapt,30-Apr-2015 13:53:51,3,1,2,scan,-10,6,b,-10,BEACH-10.WAV,peach.png,beach.png,right,31,2@,1.623,0.15,0.002,2412.0169,30-Apr-2015 14:13:35,1
7,vot_adapt,30-Apr-2015 13:53:51,3,1,2,scan,-10,7,p,40,BEES40.WAV,peas.png,bees.png,left,30,1!,1.167,0.15,0.008,2415.0169,30-Apr-2015 14:13:41,2
8,vot_adapt,30-Apr-2015 13:53:51,3,1,2,scan,-10,8,b,-10,BEES-10.WAV,peas.png,bees.png,right,31,2@,1.519,0.15,0.004,2421.0169,30-Apr-2015 14:13:44,1
9,vot_adapt,30-Apr-2015 13:53:51,3,1,2,scan,-10,9,b,-10,BEAK-10.WAV,beak.png,peak.png,left,30,1!,1.343,0.15,0.002,2424.0169,30-Apr-2015 14:13:50,2
10,vot_adapt,30-Apr-2015 13:53:51,3,1,2,scan,-10,10,b,-10,BEAK-10.WAV,peak.png,beak.png,right,31,2@,1.111,0.15,0.007,2430.0169,30-Apr-2015 14:13:53,2


In [10]:
gg = groupby(behav[end:-1:1,:], [:run_number, :subject_num])
vcat((g[1, gg.cols] for g in gg)...)

,run_number,subject_num
1,1,3
2,2,3
3,3,3
4,4,3


In [41]:
function nest(df::DataFrame, cols::Vector{Symbol}; data_col=:data)
    grouped = groupby(df, cols)
    nested = vcat((g[1, cols] for g in grouped)...)
    nested[data_col] = convert(Vector{AbstractDataFrame}, collect(grouped))
    return nested
end

nest (generic function with 1 method)

In [42]:
nested_by_run = nest(behav, [:run_number, :subject_num])
#join(run_niftis, nested_by_run, on = [:run_number, :subject_num])

4×3 DataFrames.DataFrame
│ Row │ run_number │ subject_num │ data            │
├─────┼────────────┼─────────────┼─────────────────┤
│ 1   │ 1          │ 3           │ 55×22 DataFrames.SubDataFrame{Array{Int64,1}}
│ Row │ experiment_name │ expt_start_date_time   │ subject_num │
├─────┼─────────────────┼────────────────────────┼─────────────┤
│ 1   │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 2   │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 3   │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 4   │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 5   │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 6   │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 7   │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 8   │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 9   │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 10  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 11  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
⋮
│ 44  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 45  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 46  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 47  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 48  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 49  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 50  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 51  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 52  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 53  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 54  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 55  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │

│ Row │ subject_handedness │ run_number │ scan_or_behav │ b_vot_cond │
├─────┼────────────────────┼────────────┼───────────────┼────────────┤
│ 1   │ 1                  │ 1          │ "scan"        │ -10        │
│ 2   │ 1                  │ 1          │ "scan"        │ -10        │
│ 3   │ 1                  │ 1          │ "scan"        │ -10        │
│ 4   │ 1                  │ 1          │ "scan"        │ -10        │
│ 5   │ 1                  │ 1          │ "scan"        │ -10        │
│ 6   │ 1                  │ 1          │ "scan"        │ -10        │
│ 7   │ 1                  │ 1          │ "scan"        │ -10        │
│ 8   │ 1                  │ 1          │ "scan"        │ -10        │
│ 9   │ 1                  │ 1          │ "scan"        │ -10        │
│ 10  │ 1                  │ 1          │ "scan"        │ -10        │
│ 11  │ 1                  │ 1          │ "scan"        │ -10        │
⋮
│ 44  │ 1                  │ 1          │ "scan"        │ -10        │
│ 45  │ 1                  │ 1          │ "scan"        │ -10        │
│ 46  │ 1                  │ 1          │ "scan"        │ -10        │
│ 47  │ 1                  │ 1          │ "scan"        │ -10        │
│ 48  │ 1                  │ 1          │ "scan"        │ -10        │
│ 49  │ 1                  │ 1          │ "scan"        │ -10        │
│ 50  │ 1                  │ 1          │ "scan"        │ -10        │
│ 51  │ 1                  │ 1          │ "scan"        │ -10        │
│ 52  │ 1                  │ 1          │ "scan"        │ -10        │
│ 53  │ 1                  │ 1          │ "scan"        │ -10        │
│ 54  │ 1                  │ 1          │ "scan"        │ -10        │
│ 55  │ 1                  │ 1          │ "scan"        │ -10        │

│ Row │ trial_number │ category │ vot │ word           │ picture_left │
├─────┼──────────────┼──────────┼─────┼────────────────┼──────────────┤
│ 1   │ 1            │ "p"      │ 20  │ "BEES20.WAV"   │ "bees.png"   │
│ 2   │ 2            │ "b"      │ -10 │ "BEES-10.WAV"  │ "peas.png"   │
│ 3   │ 3            │ "p"      │ 10  │ "BEACH10.WAV"  │ "peach.png"  │
│ 4   │ 4            │

# Connecting events and NIfTIs

Next step is to construct a (sparse) matrix that can transform the events model matrix to the fMRI time series.  Will be event-by-TR.  The first thing we need is a hemodynamic response function (HRF)


## Canonical HRF

SPM uses a canonical HRF that's two gamma PDFs, one for the initial rise and another for the undershoot.  The SPM implementation uses the $\alpha, \beta$ parametrization, while Julia's gamma uses $\alpha, \theta=1/\beta$.  From what I can tell, the parameters are $\alpha=6$ for the peak, $\alpha=16$ for the undershoot, and $\beta$ is set to make the scale 1 second, and the undershoot is scaled by $1/6$.

When generating the HRF, by default it generates 16 frames per TR, and then subsamples back to the correct temporal resolution.  I'm not sure why; presumably because they want to use fractional offsets, but it still doesn't make sense.

The whole thing is normalized to sum to 1, but I'm not going to bother with that...

In [13]:
using Distributions

In [14]:
immutable HRF{T,S}
    peak::Gamma{T}
    undershoot::Gamma{T}
    undershoot_scale::S
end

In [15]:
(h::HRF)(t) = pdf(h.peak, t) - pdf(h.undershoot, t) * h.undershoot_scale
(h::HRF){T<:Real}(t::T, t_offset::T) = h(t-t_offset)

In [16]:
canonical_hrf = HRF(Gamma(6.0, 1.0), Gamma(16.0, 1.0), 1/6)

# check offset
canonical_hrf.(0:3:32) == canonical_hrf.(3:3:35, 3)


true

## A single run


Let's start by looking at one run.

In [17]:
function make_tr_to_event_mat(ni::NIVolume, events::AbstractDataFrame, time_col::Symbol; hrf::HRF=canonical_hrf)
    n_t = size(ni, 4)
    TR = time_step(ni.header) / 1000
    tr_ts = TR * range(0., n_t)

    event_ts = events[time_col]
    event_ts -= minimum(event_ts)

    # HACK. specify this somewhere...in the canonical HRF?
    hrf_len_s = 32

    sparse(Float64[(event_t <= tr_t <= event_t+hrf_len_s) ? canonical_hrf(tr_t-event_t) : 0
                   for tr_t in tr_ts, event_t in event_ts])
end

events = nested_by_run[1, :data]
ni = run_niftis[1, :nifti]

tr_by_event = make_tr_to_event_mat(ni, events, :start_secs)

116×55 sparse matrix with 600 Float64 nonzero entries:
	[2  ,   1]  =  0.101658
	[3  ,   1]  =  0.159794
	[4  ,   1]  =  0.0564006
	[5  ,   1]  =  0.000158514
	[6  ,   1]  =  -0.0152137
	[7  ,   1]  =  -0.0127044
	[8  ,   1]  =  -0.00639594
	[9  ,   1]  =  -0.0023371
	[10 ,   1]  =  -0.000672901
	[11 ,   1]  =  -0.000160727
	⋮
	[114,  54]  =  -0.00424808
	[115,  54]  =  -0.00138701
	[116,  54]  =  -0.000365926
	[109,  55]  =  0.00968648
	[110,  55]  =  0.16783
	[111,  55]  =  0.113119
	[112,  55]  =  0.0241093
	[113,  55]  =  -0.010118
	[114,  55]  =  -0.0152973
	[115,  55]  =  -0.00979193
	[116,  55]  =  -0.00421316

In [18]:
using Plots
gr()

Plots.GRBackend()

In [19]:
plot(tr_by_event)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 39.4926,352.551 44.3323,141.388 49.172,20.628 54.0118,235.396 58.8515,352.221 63.6913,384.152 68.531,378.94 73.3708,365.836 78.2105,357.405 83.0502,353.948 
 87.89,352.885 92.7297,352.551 97.5695,352.551 102.409,352.551 107.249,352.551 112.089,352.551 116.928,352.551 121.768,352.551 126.608,352.551 131.448,352.551 
 136.287,352.551 141.127,352.551 145.967,352.551 150.807,352.551 155.646,352.551 160.486,352.551 165.326,352.551 170.166,352.551 175.005,352.551 179.845,352.551 
 184.685,352.551 189.525,352.551 194.364,352.551 199.204,352.551 204.044,352.551 208.884,352.551 213.723,352.551 218.563,352.551 223.403,352.551 228.243,352.551 
 233.082,352.551 237.922,352.551 242.762,352.551 247.602,352.551 252.441,352.551 257.281,352.551 262.121,352.551 266.96,352.551 271.8,352.551 276.64,352.551 
 281.48,352.551 286.319,352.551 291.159,352.551 295.999,352.551 300.839,352.551 305.678,352.551 310.518,352.551 315.358,352.551 320.198,352.551 325.037,352.551 
 329.877,352.551 334.717,352.551 339.557,352.551 344.396,352.551 349.236,352.551 354.076,352.551 358.916,352.551 363.755,352.551 368.595,352.551 373.435,352.551 
 378.275,352.551 383.114,352.551 387.954,352.551 392.794,352.551 397.634,352.551 402.473,352.551 407.313,352.551 412.153,352.551 416.993,352.551 421.832,352.551 
 426.672,352.551 431.512,352.551 436.351,352.551 441.191,352.551 446.031,352.551 450.871,352.551 455.71,352.551 460.55,352.551 465.39,352.551 470.23,352.551 
 475.069,352.551 479.909,352.551 484.749,352.551 489.589,352.551 494.428,352.551 499.268,352.551 504.108,352.551 508.948,352.551 513.787,352.551 518.627,352.551 
 523.467,352.551 528.307,352.551 533.146,352.551 537.986,352.551 542.826,352.551 547.666,352.551 552.505,352.551 557.345,352.551 562.185,352.551 567.025,352.551 
 571.864,352.551 576.704,352.551 581.544,352.551 586.384,352.551 591.223,352.551 596.063,352.551 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 39.4926,352.551 44.3323,352.551 49.172,352.551 54.0118,137.913 58.8515,22.0643 63.6913,236.89 68.531,352.75 73.3708,384.212 78.2105,378.834 83.0502,365.741 
 87.89,357.36 92.7297,353.933 97.5695,352.88 102.409,352.551 107.249,352.551 112.089,352.551 116.928,352.551 121.768,352.551 126.608,352.551 131.448,352.551 
 136.287,352.551 141.127,352.551 145.967,352.551 150.807,352.551 155.646,352.551 160.486,352.551 165.326,352.551 170.166,352.551 175.005,352.551 179.845,352.551 
 184.685,352.551 189.525,352.551 194.364,352.551 199.204,352.551 204.044,352.551 208.884,352.551 213.723,352.551 218.563,352.551 223.403,352.551 228.243,352.551 
 233.082,352.551 237.922,352.551 242.762,352.551 247.602,352.551 252.441,352.551 257.281,352.551 262.121,352.551 266.96,352.551 271.8,352.551 276.64,352.551 
 281.48,352.551 286.319,352.551 291.159,352.551 295.999,352.551 300.839,352.551 305.678,352.551 310.518,352.551 315.358,352.551 320.198,352.551 325.037,352.551 
 329.877,352.551 334.717,352.551 339.557,352.551 344.396,352.551 349.236,352.551 354.076,352.551 358.916,352.551 363.755,352.551 368.595,352.551 373.435,352.551 
 378.275,352.551 383.114,352.551 387.954,352.551 392.794,352.551 397.634,352.551 402.473,352.551 407.313,352.551 412.153,352.551 416.993,352.551 421.832,352.551 
 426.672,352.551 431.512,352.551 436.351,352.551 441.191,352.551 446.031,352.551 450.871,352.551 455.71,352.551 460.55,352.551 465.39,352.551 470.23,352.551 
 475.069,352.551 479.909,352.551 484.749,352.551 489.589,352.551 494.428,352.551 499.268,352.551 504.108,352.551 508.948,352.551 513.787,352.551 518.627,352.551 
 523.467,352.551 528.307,352.551 533.146,352.551 537.986,352.551 542.826,352.551 547.666,352.5

In [20]:
heatmap(tr_by_event)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
<image width="505" height="381" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAfkAAAF9CAYAAAAHlxEqAAARpklEQVR4nO3dXezed1UA8PPvOra2
63u7ritbuNI7E280Ck6o0K3v7+3aruvs2AqK22ADGeylVUARpwkGEjFsujlxDjZgM26REE3UCxIu
vPJGTQwxgoCNY/++/Nv+68WuvNjOSfbj4XnO8/lcnzy/38X+Oz3nfL/nN7Nx8YnLAQCMjUpinouL
acyCN/8qAMA4kuQBoClJHgCaWviTfgEA4P+7XJjKn4lzaYxKHgCakuQBoCntegAYMxfiUhozu+BM
GqOSB4CmJHkAaEq7HgBGqLLNbrZwcn7x/OI0RiUPAE1J8gDQlCQPAE2ZyQPAQCrz9tI2u8L1uF9Z
ckMao5IHgKYkeQBoSrseAAqGasXPFbbZVa7Hndr2jTRGJQ8ATUnyANCUdj0AU63Shn8tbphWfOU7
8L+4dHUas/yuq9IYlTwANCXJA0BT2vUATLVKGz5iuFZ8ZdHNqR3fTmMOb3pPGqOSB4CmJHkAaEqS
B4CmzOQBmGqVWXvEcPP2JYVtdsuO59fjTj89l8ao5AGgKUkeAJrSrgegrcrluEobPmK4Vnxlm92h
m3+p8EYX0giVPAA0JckDQFPa9QC0VdlmV2nDR0QsmV+Sxrx92ao05uTOl9OYA0+9s/JKKZU8ADQl
yQNAU5I8ADRlJg/ARKpcj6tss6vM2iOK8/ZdL6Uxy46+JX/YU5U3yqnkAaApSR4AmtKuB2DsVFrx
petxhW1271iWb6CLiDi1+2/SmKVH8lb8gc3vLDwt32ZXoZIHgKYkeQBoSrsegLEzX2jFXyicnK9s
szu1+1uld1p65Mo05sDmd6Uxr1waphVfoZIHgKYkeQBoSrsegJGqnJwvteILJ+cri26WHs7b8BER
+zdvTGN+NMJWfIVKHgCakuQBoClJHgCaMpMHYKQq1+Mq8/bK9bhbrrkhjdm/JZ+1R4zfvL1CJQ8A
TUnyANCUdj0Ag6hcjYsYblPdNYXrcSf3/XUas+/xd6cxk0olDwBNSfIA0JR2PQCpSiu+cmo+IuJM
nE1jKq34dyxflf/O/kKaezwPmVQqeQBoSpIHgKYkeQBoykweYMoNNW+vXI2LiDizIJ/Jb16ab6o7
ue/FNGbvts2FN7pYiJlMKnkAaEqSB4CmtOsBptxQrfjK1biIiGvmr0ljTu5/IY1ZsjtPYbNf7NuK
r1DJA0BTkjwANKVdDzDlhmrFV07NR0RsKZycX7LrijRmz45Nhadp1wMADUnyANCUdj1AY5VFN0O1
4iun5iMiTh78ehqzZ8fW/J3mp7sVX6GSB4CmJHkAaEqSB4CmzOQBGqtssxtq3v7Ly1eW3mnJ9vx6
3JkvmLcPQSUPAE1J8gDQlHY9wISqXI+rbLNbWmjF37Qib8WfPPi1whtF7Nq5rRClXT8ElTwANCXJ
A0BT2vUAE6pycn62sM1u44o1aczJW7+axizenJ+aj4g4+8da8aOikgeApiR5AGhKkgeApszkAcbQ
UNfjzha22Z069Hwas+jmfN6+a/eWNOY1ZvKjopIHgKYkeQBoSrseYAwNdT1u6fzSNGbRprze27kn
b8Wfm9eGHzcqeQBoSpIHgKa06wFGqHJqPmK4k/Pbl92QxuzcuzWNOTefvw/jRyUPAE1J8gDQlHY9
wAhVTs1HDHdy/tEjX0ljdn2u8n13JpFKHgCakuQBoClJHgCaMpMHGKHK1biI2vW4HYXrcYveNZM/
7HOVN2ISqeQBoClJHgCa0q4HGEjlclzlalxExLLK9bijX05jdhzYVXiabXZdqeQBoClJHgCa0q4H
GEhlm13l1HxExKbl+cn5q2/Kn3f+j7Tip5lKHgCakuQBoClJHgCaMpMHKKhcj6tss1s2v6z0vEeO
PpvGbD+wu/BLZvLTTCUPAE1J8gDQlHY9QEHlelxlm92mlatLz1v09vk0Zu6zWvG8MZU8ADQlyQNA
U9r1AAWVk/OVbXaPHvur0vO23bq3EKVdzxtTyQNAU5I8ADSlXQ9MvaG+A19ZdHP1z+en5iMi5i5r
xfPmqeQBoClJHgCakuQBoCkzeaC1yry9ss2ucj3ulhX5Nruth/cX3ijC9TiGoJIHgKYkeQBoSrse
aK3Sip8rtMaXF67HPfKrz6Qx2x+rfAMehqGSB4CmJHkAaEq7HmhtLi6mMWcK2+w2r1qTxlz1s07E
M15U8gDQlCQPAE1J8gDQlJk8MJEqm+wiavP2swvOpTEPH/9SGrP16IHCG9W+QgdDUMkDQFOSPAA0
pV0PTKTKJruIWit++fzyNOaqn8mvx124rBXPeFHJA0BTkjwANDWzcfGJ6iFVgLFxNi6U4i4WTrO/
e9XSNObbp/PNeRe16xkzKnkAaEqSB4CmnK4Hxk5lhjhbWHITEbFtdf5hmYfe+3T+O5/eX3oejBOV
PAA0JckDQFOSPAA0ZSYPjJ3KNrtzhU12EREP35XP26/86fx6HEwilTwANCXJA0BT2vXA2JmLvH2+
4lL+UZmIiCt/Kv+tzcePFH7JNjsmj0oeAJqS5AGgKe16YKSG2ma3fU2+yS6i1oq/pBVPUyp5AGhK
kgeApiR5AGjKTB4YqaG22T104qnS87Z+6mApDjpSyQNAU5I8ADSlXQ+MVG2b3Yo05sq3XRjidaA1
lTwANCXJA0BT2vXAYIbaZrdjbb7NbvPdtxeeFuHDMkwzlTwANCXJA0BT2vVASaUVP9iim/c9mcZs
+cSthTeC6aaSB4CmJHkAaEqSB4CmzOSBksq8vbLNbmVhm93CDedL7wS8MZU8ADQlyQNAUzMbF5+o
3IwBGqv8T+Bc5B+EmZ3Jt9ntWr06jfmnH1xKY+ZtsoOUSh4AmpLkAaApp+uB0sn5Siv+3Ey+ze5j
v/ZnacyW3zqcxgA5lTwANCXJA0BTkjwANGUmD5Q21VXm7Svn8212V6zNr+IBw1DJA0BTkjwANGXj
HTRW/eP+4cwracyiy1enMbesXZLG/OP389FA5UofkFPJA0BTkjwANOV0PTRWbXtXTs7vWZN/WObB
D/xpGrP50SOVVwIGoJIHgKYkeQBoSrseGqssuYmIWDW/Mo158ANPpDELluffgQdGRyUPAE1J8gDQ
lCQPAE2ZycOEqlyOm505U/qtvWvWpDELluXz9s333VF4mm12MCoqeQBoSpIHgKa062FCVbbZnZs5
X/qtj97zeBpzywfvSGMua8XDWFHJA0BTkjwANKVdD2Oo0vSubLOrbLKLiFiwaD6N0YqHyaOSB4Cm
JHkAaEqSB4CmZjYuPmHQBiNU+YOrXI87PfNKGrNv7erC0yL+/r/zbXZm8jB5VPIA0JQkDwBNuUIH
A6q14vPranORt88r2+w+em++yS4i4u8+fnspDpgsKnkAaEqSB4CmtOthQEO14l8tfAd+dWWb3UIn
4mGaqeQBoClJHgCa0q6HAZ0vtOJnC63484WT8wcLi242/+YdacxrtPWhI5U8ADQlyQNAU5I8ADRl
Jg8F1Yn1UPP2yvW4j3zwi2nMNx88lsYAfankAaApSR4AmtKuZ+oN9VGZiOFa8dvWLcofNj9TeSVg
iqnkAaApSR4AmtKuZ+pVWvGVTXYRw7XiP/yh/OR8bZudTXYwzVTyANCUJA8ATUnyANCUmTxTb6gv
x0VEHLo2/zLcA/fn8/bL5/N/f182bwcSKn

In [21]:
pool!(behav, :category)

mm = ModelMatrix(ModelFrame(Formula(nothing, :(0 + category)), events))
mm.m

55×2 Array{Float64,2}:
 0.0  1.0
 1.0  0.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 1.0  0.0
 0.0  1.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 ⋮       
 0.0  1.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 1.0  0.0
 0.0  1.0
 0.0  1.0
 1.0  0.0
 1.0  0.0
 0.0  1.0
 1.0  0.0
 0.0  1.0

In [22]:
plot(tr_by_event * mm.m)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,357.699 35.5207,357.699 40.4377,357.699 45.3548,238.041 50.2718,173.457 55.1889,293.22 60.1059,357.81 65.0229,375.349 69.94,372.351 74.857,365.052 
 79.774,360.379 84.6911,231.164 89.6081,176.965 94.5252,296.497 99.4422,228.832 104.359,195.844 109.276,180.129 114.193,54.8575 119.11,140.584 124.027,181.156 
 128.944,207.447 133.861,334.334 138.779,244.538 143.696,209.191 148.613,318.245 153.53,228.088 158.447,203.572 163.364,316.782 168.281,225.031 173.198,204.795 
 178.115,317.859 183.032,368.183 187.949,377.999 192.866,225.093 197.783,193.575 202.7,158.462 207.617,42.7503 212.534,3.93701 217.451,155.226 222.368,183.451 
 227.285,228.375 232.202,346.141 237.119,384.952 242.036,229.656 246.953,208.31 251.87,315.971 256.787,365.135 261.704,217.713 266.621,207.269 271.539,316.523 
 276.456,365.748 281.373,376.365 286.29,370.562 291.207,200.71 296.124,198.914 301.041,312.895 305.958,364.349 310.875,374.892 315.792,203.934 320.709,205.428 
 325.626,316.124 330.543,364.175 335.46,206.209 340.377,214.37 345.294,319.938 350.211,195.512 355.128,221.608 360.045,327.617 364.962,197.685 369.879,224.638 
 374.796,328.592 379.713,196.133 384.63,226.191 389.547,331.53 394.464,369.383 399.381,199.618 404.299,221.375 409.216,325.445 414.133,368.411 419.05,376.237 
 423.967,370.237 428.884,363.298 433.801,359.587 438.718,358.21 443.635,357.815 448.552,357.699 453.469,357.699 458.386,357.699 463.303,357.699 468.22,357.699 
 473.137,357.699 478.054,357.699 482.971,357.699 487.888,357.699 492.805,357.699 497.722,350.749 502.639,168.584 507.556,219.637 512.473,326.068 517.39,368.374 
 522.307,375.558 527.224,360.693 532.142,162.076 537.059,32.5116 541.976,194.003 546.893,339.361 551.81,376.269 556.727,193.422 561.644,242.169 566.561,335.871 
 571.478,371.337 576.395,375.969 581.312,369.198 586.229,362.618 591.146,359.305 596.063,358.122 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,357.699 35.5207,239.978 40.4377,172.656 45.3548,292.386 50.2718,357.515 55.1889,253.731 60.1059,188.982 65.0229,301.454 69.94,236.349 74.857,68.5643 
 79.774,128.293 84.6911,303.821 89.6081,250.809 94.5252,210.218 99.4422,318.779 104.359,368.911 109.276,378.837 114.193,373.006 119.11,364.993 124.027,360.267 
 128.944,358.431 133.861,357.872 138.779,357.699 143.696,357.699 148.613,357.699 153.53,357.699 158.447,357.699 163.364,357.699 168.281,357.699 173.198,357.699 
 178.115,357.649 183.032,212.236 187.949,185.837 192.866,304.24 197.783,213.578 202.7,205.25 207.617,319.246 212.534,368.624 217.451,378.04 222.368,372.068 
 227.285,364.413 232.202,359.806 237.119,203.666 242.036,191.34 246.953,308.386 251.87,362.645 256.787,217.864 261.704,206.275 266.621,156.758 271.539,201.691 
 276.456,168.235 281.373,214.643 286.29,335.568 291.207,379.33 296.124,382.582 301.041,372.464 305.958,199.028 310.875,200.565 315.792,312.987 320.709,197.083 
 325.626,217.908 330.543,327.381 335.46,370.894 340.377,377.836 345.294,371.196 350.211,363.792 355.128,359.744 360.045,358.259 364.962,357.827 369.879,357.699 
 374.796,357.699 379.713,355.511 384.63,181.462 389.547,206.695 394.464,318.671 399.381,366.192 404.299,372.84 409.216,190.957 414.133,215.033 419.05,318.813 
 423.967,186.211 428.884,225.371 433.801,151.471 438.718,222.759 443.635,158.508 448.552,230.829 453.469,161.788 458.386,237.881 463.303,341.803 468.22,190.385 
 473.137,47.7489 478.054,9.8787 482.971,193.282 487.888,166.836 492.805,251.951 497.722,171.772 502.639,248.459 507.556,343.614 512.473,192.454 517.39,244.373 
 522.307,331.828 527.224,182.894 532.142,242.006 537.059,330.242 541.976,181.511 546.893,243.477 551.81,340.416

In [23]:
ni[32, 32, 15, :]

116-element Array{Float32,1}:
 478.0
 468.0
 469.0
 466.0
 465.0
 474.0
 462.0
 468.0
 463.0
 461.0
 465.0
 468.0
 466.0
   ⋮  
 453.0
 439.0
 446.0
 452.0
 452.0
 445.0
 445.0
 449.0
 449.0
 453.0
 455.0
 455.0

In [24]:
using GLM

In [25]:
X = [ones(size(tr_by_event, 1)) tr_by_event * mm.m]
y = convert(Vector{Float64}, ni[32, 32, 15, :])

fitted = lm(X, y)


GLM.LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}}:

Coefficients:
     Estimate Std.Error   t value Pr(>|t|)
x1    452.829   1.35432   334.359   <1e-99
x2    27.0898   10.2266   2.64895   0.0092
x3   -4.15423    10.475 -0.396585   0.6924



In [26]:
plot([y predict(fitted)])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 440 
 
 
 450 
 
 
 460 
 
 
 470 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,3.93701 35.5207,101.633 40.4377,91.8635 45.3548,121.172 50.2718,130.942 55.1889,43.0155 60.1059,160.251 65.0229,101.633 69.94,150.481 74.857,170.02 
 79.774,130.942 84.6911,101.633 89.6081,121.172 94.5252,170.02 99.4422,150.481 104.359,101.633 109.276,199.329 114.193,91.8635 119.11,189.56 124.027,160.251 
 128.944,91.8635 133.861,218.868 138.779,199.329 143.696,101.633 148.613,228.638 153.53,277.486 158.447,218.868 163.364,228.638 168.281,218.868 173.198,218.868 
 178.115,228.638 183.032,170.02 187.949,199.329 192.866,257.947 197.783,199.329 202.7,170.02 207.617,150.481 212.534,101.633 217.451,150.481 222.368,189.56 
 227.285,228.638 232.202,218.868 237.119,248.177 242.036,121.172 246.953,170.02 251.87,199.329 256.787,257.947 261.704,248.177 266.621,160.251 271.539,277.486 
 276.456,316.565 281.373,257.947 286.29,257.947 291.207,257.947 296.124,316.565 301.041,345.873 305.958,345.873 310.875,326.334 315.792,287.256 320.709,257.947 
 325.626,345.873 330.543,287.256 335.46,257.947 340.377,277.486 345.294,287.256 350.211,287.256 355.128,257.947 360.045,257.947 364.962,248.177 369.879,277.486 
 374.796,257.947 379.713,345.873 384.63,316.565 389.547,345.873 394.464,287.256 399.381,297.025 404.299,297.025 409.216,248.177 414.133,218.868 419.05,257.947 
 423.967,297.025 428.884,345.873 433.801,297.025 438.718,287.256 443.635,297.025 448.552,287.256 453.469,297.025 458.386,287.256 463.303,287.256 468.22,228.638 
 473.137,355.643 478.054,355.643 482.971,199.329 487.888,287.256 492.805,228.638 497.722,218.868 502.639,257.947 507.556,287.256 512.473,345.873 517.39,316.565 
 522.307,316.565 527.224,170.02 532.142,297.025 537.059,287.256 541.976,248.177 546.893,384.952 551.81,316.565 556.727,257.947 561.644,257.947 566.561,326.334 
 571.478,326.334 576.395,287.256 581.312,287.256 586.229,248.177 591.146,228.638 596.063,228.638 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e9746a; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,249.849 35.5207,253.975 40.4377,256.335 45.3548,224.791 50.2718,207.748 55.1889,238.757 60.1059,255.788 65.0229,255.855 69.94,257.451 74.857,261.664 
 79.774,258.502 84.6911,222.819 89.6081,212.29 94.5252,241.031 99.4422,221.762 104.359,212.465 109.276,208.526 114.193,180.1 119.11,199.973 124.027,209.411 
 128.944,215.484 133.861,244.504 138.779,223.987 143.696,215.909 148.613,240.832 153.53,220.228 158.447,214.625 163.364,240.498 168.281,219.529 173.198,214.904 
 178.115,240.746 183.032,257.344 187.949,260.512 192.866,221.417 197.783,217.391 202.7,209.658 207.617,179.217 212.534,168.616 217.451,202.862 222.368,209.522 
 227.285,220.058 232.202,247.134 237.119,261.477 242.036,226.416 246.953,217.436 251.87,240.14 256.787,256.45 261.704,223.163 266.621,222.512 271.539,245.907 
 276.456,258.329 281.373,259.13 286.29,253.565 291.207,213.212 296.124,212.688 301.041,239.092 305.958,256.931 310.875,259.286 315.792,216.274 320.709,220.678 
 325.626,245.247 330.543,252.392 335.46,214.765 340.377,216.387 345.294,240.746 350.211,212.569 355.128,218.675 360.045,242.955 364.962,213.275 369.879,219.439 
 374.796,243.197 379.713,213.001 384.63,225.971 389.547,249.161 394.464,253.888 399.381,213.423 404.299,218.163 409.216,248.322 414.133,257.298 419.05,255.449 
 423.967,258.725 428.884,255.767 433.801,257.509 438.718,254.696 443.635,256.857 448.552,254.296 453.469,256.716 458.386,254.049 463.303,250.407 468.22,255.713 
 473.137,260.712 478.054,262.04 482.971,255.612 487.888,256.539 492.805,253.556 497.722,254.778 502.639,210.457 507.556,218.79 512.473,248.412 517.39,256.261 
 522.307,254.838 527.224,256.66 532.142,209.195 537.059,176.492 541.976,218.612 546.893,249.662 551.81,254.699 55

### Sparse matrix

Using a sparse matrix for the transformation is a lot faster (which would hopefully be the case given that it's mostly zeros...)

In [27]:
#event_to_tr = Float64[event_t <= tr_t ? canonical_hrf(tr_t-event_t) : 0 for tr_t in tr_ts, event_t in event_ts]
using BenchmarkTools

@benchmark $(full(tr_by_event)) * $(mm.m)

BenchmarkTools.Trial: 
  memory estimate:  1.98 KiB
  allocs estimate:  1
  --------------
  minimum time:     7.204 μs (0.00% GC)
  median time:      7.564 μs (0.00% GC)
  mean time:        7.955 μs (1.89% GC)
  maximum time:     520.711 μs (97.36% GC)
  --------------
  samples:          10000
  evals/sample:     4
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [28]:
@benchmark $(tr_by_event) * $(mm.m)

BenchmarkTools.Trial: 
  memory estimate:  1.98 KiB
  allocs estimate:  1
  --------------
  minimum time:     1.481 μs (0.00% GC)
  median time:      1.545 μs (0.00% GC)
  mean time:        1.775 μs (7.80% GC)
  maximum time:     178.814 μs (98.18% GC)
  --------------
  samples:          10000
  evals/sample:     10
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [29]:
# row or column major??
full(mm.m' * tr_by_event')' == tr_by_event * mm.m
@benchmark full($(mm.m') * $(tr_by_event'))'

BenchmarkTools.Trial: 
  memory estimate:  3.97 KiB
  allocs estimate:  2
  --------------
  minimum time:     1.910 μs (0.00% GC)
  median time:      2.006 μs (0.00% GC)
  mean time:        2.329 μs (8.51% GC)
  maximum time:     158.959 μs (97.79% GC)
  --------------
  samples:          10000
  evals/sample:     10
  time tolerance:   5.00%
  memory tolerance: 1.00%

## Multiple runs

Generate separate transformers for each run, then combine to create a transformer for all runs.

In [30]:
tr_to_ev = blkdiag(make_tr_to_event_mat.(run_niftis[:nifti], nested_by_run[:data], [:start_secs], hrf=canonical_hrf)...)
heatmap(tr_to_ev)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 50 
 
 
 100 
 
 
 150 
 
 
 200 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
<image width="505" height="381" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAfkAAAF9CAYAAAAHlxEqAAAgAElEQVR4nO3dWZRd1X3n8e8tzSVV
SaUBzcQPeUi/9Oq1suKkbbCxMFWqklSa5xExCGwQGAuwMCBhPGOcxFnO4JjJgMGMtrFjsB3aZLDT
6aSddBK70+PquDvYCUgCpJJqutUP5+xzji4lqYZb507fT1aWbp29L2wvQD/9995n78KK5gNDSJKk
ulEkivbJFR6HJEkqs34GAUNekqS6EabmT3EGMOQlSaobQ3HM9zT1AIa8JEl1oy+epm8uNgOGvCRJ
NWnorM/RT681HQPgz7b9V8CQlySppgwlv6YxX1rBz756GgAFX6GTJKl2hNfjQrAD9MQb7d7dMg+A
V0/7Cp0kSTUnhHsIdkg32t2z9q8A2PLYZYAhL0lSTQjT7iHcQ7ADzCzOBKBl39TowWPRL4a8JElV
LIR7LwMAFCgA0D5redLnnvXfAaDz/SvO+q4hL0lSlclulgtr8KGCv7Q1Wnc/Ggc7QMvOKdGHR87+
6xjykiRVmWIm5sMRtWF6/uiG/whAy/apSZ/NnSuS3lmGvCRJVaZ/mJ3zybr7tqhq39yVTs2/NXh2
uAeGvCRJVWKw5FAbgFlxuP/xVT8EYOXKK+KW4YM9y5CXJKlKhAp+VnFW8uzS2W0AzNwYR/YDI//r
GfKSJFVY+nrcaQDeO3t+0nZ08wsAbFzTFT8ZGPFf15CXJKkCht1B3xSF/D1bvpm0Na+NovrUl0ce
7oEhL0lSjkK4D7eDPkzTz+yelLSt7+6IPxnykiRVpbCpLgT6qXhqHuB0XMH/p2v+MwAda1Zmvjn6
cA8MeUmSclAa7iHYAVriCr65qyl68Afl+Xsa8pIkTaAwKV8a7i3FlqTPe+fMAWDd+tFvrjsfQ16S
pDIbblNdabiHYAc4uuN5ANb9bhflZMhLkjROpZvpsifWhQp+TWt0ocyRnc8CMGNFU9Jn7eY18af0
e+VgyEuSNE6l4T7cprqju6Jwn35ZdItcdxLs0Fssb7gHhrwkSeN0vk11rfH0/PTLoj8IdG9ZC0xc
sGcZ8pIkjdG5NtW1ZjbVva8tWnvv3rwOgN6hiQ/3wJCXJGkUzreprrXYCsD72lqTPnfvfhqAtV/o
zmeAGYa8JEmj0Jt5vS1U8OtmLwPg7r1PATD9N9I/Cqzevin+lF8FHxjykiSNQOnUPKQV/N374nB/
ZxGAVds3J336c5yeL2XIS5I0AqVT85BOz0//tTjcd0ThXslgzyqsaD4wdOFukiQ1phCSZ+Jp+oHM
dP2KuVHI/83xKNSrJdwDK3lJks4juQaWHgA656Z3vd915ZMArLl/Q/4DGwFDXpKkEtkp7hO8BcBU
pgHw8d96LGnr2rst/lTMaWSjY8hLklQie9f76aYzAHTNiSr4NNhhYKg6wz0w5CVJioVo78usu88u
zgbgrmseB2D1ZzaXfq1qGfKSJMVCBZ99TW71vKiCn/or5bn+NU+GvCSp4YUK/ni8/j59aHrS9rHP
PAxA+5U7ch7V+BnykqSGFyr4M/H6+5q2dAd959W7kl61xpCXJDWs0jX4OYPRZTJ3Xvdo0mfVJ7bm
PayyMeQlSQ2rdA2+e0FUwU+5uK9iYyonQ16S1FCy78CHCj5M09953VcA6DywN9Or9qbpA4+1lSQ1
lNP0J59fbzoOwLa5SwH44WvRsbTFGg72LCt5SVJDSG6RK6Svx7UVozX4Oz7wCABdH6u9HfTnY8hL
kupaCPdkB33hTNK2ft48ACYt6C/9Wl0w5CVJdS2Ee1h/D9U7wOEbHgag88Z98ZP6WsE25CVJdWkw
DuzjheiAm1DB/92R7yZ9Vh3ZDZx9Vn09MeQlSXUp2Tkfh/vcYhsATa3pne/1Gu6Bu+slSXUjG2iv
F94AoHmoGYDOBdGvf/KvA5n+9R2BVvKSpLqRrczPFHoB2DQ/OuDm9pseBOCVO3fnP7AKMeQlSTVv
uCtiw/T87Tc9AEBhWn1X7cMx5CVJNWswPrSmj2idPRxuA/D3H/8OAJ137APqf2p+OIa8JKmmZKO6
Nw73U4UeAObF1TsAhdC/8cI9MOQlSTUhPdQmPXI2hHtvvP6+dcG8pK3z8L6SbzYeQ16SVBNCuIfq
HdJwDxX8bbc8kLS9/JHsJTONyZCXJFW1UIeXTs1DGu6rF86I+g4Uch1btTPkJUlVKYR7uDXuWLyp
bn5xbtLnB/c9HvU9PQmAzjv35DfAGmDIS5KqUpieD5V7CPc1C6clfYZ6ohjrOrIrPMlvgDXAkJck
VY3hds6HdfcdF0Uhf+jWB5M+Kw/ti79nuA/HkJckVY3hds6HCv7QbVG4D74xJeljuJ+fIS9Jqrhw
qM2x+MY4gNahmQD84DejdfeOD+2LWwz2kTLkJUkVVzo1D9C9MKrgB14La/CG+2gZ8pKkigmxHabm
F2R2zt9yOJqe77hpX0lvjZQhL0nKXenpdaGCX7soDfmBn88o6a3RMuQlSbkrPb1uQTE6jvZDh9Od
8x0HPbFuvAx5SVIuBjI7548X3gSgt9AHwD98IboxruMGg72cDHlJ0oRK73rPnjkfhftFcQXf979b
8h5WQzDkJUkTajA5ue5U8iyE+/rF0Tvva+7fkP/AGoAhL0maEKUVfKjeATbE4X7wzocA+PYHnKaf
CIa8JKlssvvgSy+WuX7RwqTt6k1fB6DdcJ9QhrwkadxCuA9mNtedjKfnF8ZT8zceeSRpaz/gbXF5
MOQlSWNWGu7ZzXV98fT8xsVR1PT83VyUL0NekjRmabgPAHAyPrkOYGFxPgA3HH0UgPZrd6F8GfKS
pDErDfe+zOa6TUsmAdDzYyv4SjHkJUmjFg62OdZ0AoBFcdW+aUlT0mfPquiAm3W/15Xz6BQY8pKk
EcnunO+NK/gQ7puXRuF+/ceeSPq0X7U9t7FpeIa8JGlEsjvnw61xu5bOAdJwf+uH8/IfmM7JkJck
jUio3iFde7/+3icBeOtPo4p+44NX5D8wnZMhL0k6r9I73wEWx9P0b/5JvBb/0PvzHpZGwJCXJCWy
6+6DyXWwUQU/b2h20vbHjzwNQPveLbmNTaNnyEuSEtl19xDuoYLfuywN+RPfX5DvwDQmhrwkKZFd
dw+3xvUVojPor/3kM0lb++7N+Q5MY2LIS5Iy6+7pdbAh3JcUo6r9+IsLS7+mKmfIS1IDGyhZd58/
NCdp27Y8iv6rP/UcAO07N+Y8Oo2XIS9JDSxdd48q+P3L03X3qz4dXQf72jcW5T8wlYUhL0kNKEzP
h+tg++Op+as+8/Wkz2vPR+G+46l35To2lY8hL0kNKOyiD+G+tHgRAP/63OKkz86n/33+A1NZGfKS
1IDCNH0I9+0XR6FvsNcXQ16SGsRA5h344/HtcX/+5I8A+PkTUQX/5D+9M/+BacIY8pLUILLvwC+L
K/hXv7oEgL1f/7WKjEkTy5CXpAZxMvMO/LUXTwIM93pnyEtSgxiIN9kB7Ln/OwA8vml1pYajHBjy
ktQgbr94bvK5w3BvCIa8JDWI3Z//bvL5sQ1dFRyJ8mLIS1KD+NkfLq30EJSzwormA0MX7iZJkmqN
lbwk1YGhsz5HP71BtJv+VFN0H3x4bU6Nw5CXpDrQl70HnjNnta1oXg7AfzvZm+uYVHmGvCTVsOQe
+Eyw98SV++VxuB9d/ccAbH/yklzHpsoz5CWphoWp+RDsAM3FZgCOrv

In [31]:
f = Formula(nothing, :(0+category))
mm_full = reduce(vcat, ModelMatrix(ModelFrame(f, x)).m for x in nested_by_run[:data])

222×2 Array{Float64,2}:
 0.0  1.0
 1.0  0.0
 0.0  1.0
 0.0  1.0
 0.0  1.0
 1.0  0.0
 0.0  1.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 ⋮       
 1.0  0.0
 1.0  0.0
 0.0  1.0
 0.0  1.0
 1.0  0.0
 0.0  1.0
 1.0  0.0
 0.0  1.0
 1.0  0.0
 1.0  0.0
 0.0  1.0
 0.0  1.0

In [32]:
ModelMatrix(ModelFrame(f, behav)).m == mm_full

true

In [33]:
k = [:run_number, :subject_num]
join(run_niftis[reverse(1:end), k], nested_by_run[reverse(1:end), :], on=k, kind=:semi)

,run_number,subject_num
1,1,3
2,2,3
3,3,3
4,4,3


In [34]:

join(nested_by_run, run_niftis[reverse(1:end), k], on=k)

4×3 DataFrames.DataFrame
│ Row │ run_number │ subject_num │ data            │
├─────┼────────────┼─────────────┼─────────────────┤
│ 1   │ 1          │ 3           │ 55×22 DataFrames.SubDataFrame{Array{Int64,1}}
│ Row │ experiment_name │ expt_start_date_time   │ subject_num │
├─────┼─────────────────┼────────────────────────┼─────────────┤
│ 1   │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 2   │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 3   │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 4   │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 5   │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 6   │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 7   │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 8   │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 9   │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 10  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 11  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
⋮
│ 44  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 45  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 46  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 47  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 48  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 49  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 50  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 51  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 52  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 53  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 54  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │
│ 55  │ "vot_adapt"     │ "30-Apr-2015 13:53:51" │ 3           │

│ Row │ subject_handedness │ run_number │ scan_or_behav │ b_vot_cond │
├─────┼────────────────────┼────────────┼───────────────┼────────────┤
│ 1   │ 1                  │ 1          │ "scan"        │ -10        │
│ 2   │ 1                  │ 1          │ "scan"        │ -10        │
│ 3   │ 1                  │ 1          │ "scan"        │ -10        │
│ 4   │ 1                  │ 1          │ "scan"        │ -10        │
│ 5   │ 1                  │ 1          │ "scan"        │ -10        │
│ 6   │ 1                  │ 1          │ "scan"        │ -10        │
│ 7   │ 1                  │ 1          │ "scan"        │ -10        │
│ 8   │ 1                  │ 1          │ "scan"        │ -10        │
│ 9   │ 1                  │ 1          │ "scan"        │ -10        │
│ 10  │ 1                  │ 1          │ "scan"        │ -10        │
│ 11  │ 1                  │ 1          │ "scan"        │ -10        │
⋮
│ 44  │ 1                  │ 1          │ "scan"        │ -10        │
│ 45  │ 1                  │ 1          │ "scan"        │ -10        │
│ 46  │ 1                  │ 1          │ "scan"        │ -10        │
│ 47  │ 1                  │ 1          │ "scan"        │ -10        │
│ 48  │ 1                  │ 1          │ "scan"        │ -10        │
│ 49  │ 1                  │ 1          │ "scan"        │ -10        │
│ 50  │ 1                  │ 1          │ "scan"        │ -10        │
│ 51  │ 1                  │ 1          │ "scan"        │ -10        │
│ 52  │ 1                  │ 1          │ "scan"        │ -10        │
│ 53  │ 1                  │ 1          │ "scan"        │ -10        │
│ 54  │ 1                  │ 1          │ "scan"        │ -10        │
│ 55  │ 1                  │ 1          │ "scan"        │ -10        │

│ Row │ trial_number │ category │ vot │ word           │ picture_left │
├─────┼──────────────┼──────────┼─────┼────────────────┼──────────────┤
│ 1   │ 1            │ "p"      │ 20  │ "BEES20.WAV"   │ "bees.png"   │
│ 2   │ 2            │ "b"      │ -10 │ "BEES-10.WAV"  │ "peas.png"   │
│ 3   │ 3            │ "p"      │ 10  │ "BEACH10.WAV"  │ "peach.png"  │
│ 4   │ 4            │

In [35]:
Base.cat(catdim::Integer, X::NIVolume...) = NIVolume(X[1].header, cat(catdim, (ni.raw for ni in X)...))

nifti_combined = cat(4, run_niftis[:nifti]...)
@assert size(nifti_combined, 4) == sum(size(ni, 4) for ni in run_niftis[:nifti])

In [36]:
headers = [ni.header for ni in run_niftis[:nifti]]
getaffine.(headers)

4-element Array{Array{Float64,2},1}:
 [-4.0 -1.00259e-21 7.86008e-21 130.034; 2.27145e-20 -4.0 -7.2096e-20 155.757; 1.90928e-20 6.87114e-20 4.0 -57.6159; 0.0 0.0 0.0 1.0]        
 [-3.99997 0.000303863 0.0165243 129.695; -0.000619086 -3.99927 -0.0763172 156.701; 0.0165155 -0.0763191 3.99924 -54.5768; 0.0 0.0 0.0 1.0]  
 [-3.99993 0.0230993 0.000149058 128.603; -0.0230996 -3.99969 -0.044128 157.424; -0.000105785 -0.0441281 3.99976 -56.4733; 0.0 0.0 0.0 1.0]  
 [-3.99999 0.00438923 -0.00662023 129.765; -0.00436476 -3.99997 -0.0147656 156.017; -0.00663638 -0.0147584 3.99997 -56.5732; 0.0 0.0 0.0 1.0]

...of course they're different. it's the non-preprocessed data, not normalized.  this is fine for now but I'll want to add a check in the real thing.

In [37]:
nested_by_run[k]
run_niftis[k]

zip((nested_by_run[kk] for kk in k)...)

Base.Zip2{DataArrays.DataArray{Int64,1},DataArrays.DataArray{Int64,1}}([1,2,3,4],[3,3,3,3])

# Frame type

This type holds the events DataFrame, the BOLD images/NIVolumes, and the mapper between them.  Define methods on this field to create model matrices and fit models with a formula.

## Fields

* event_time_col::Symbol column in events with the event times (in seconds)
* events::Vector{AbstractDataFrame} Events for each NIVolume, with column eventtimecol
* nivolumes::Vector{NIVolume} Vector of NIVolumes
* tr_to_event::Matrix transforms between events and TRs (e.g., the HRF for each event)

In [ ]:
type BOLDFrame
    events::AbstractVector{AbstractDataFrame}
    nivolumes::AbstractVector{NIVolume}
    event_time_col::Symbol
    tr_to_event::Matrix
end



In [57]:

function BOLDFrame{T,U,V}(events::AbstractVector{AbstractDataFrame}, nivolumes::AbstractVector{NIVolume{T,U,V}}; 
                          event_time_col::Symbol=:time, hrf::HRF=canonical_hrf)
    length(events) == length(nivolumes) || error("Mismatch in number of volumes and events")
    tr_to_event = blkdiag(make_tr_to_event_mat.(nivolumes, events, [event_time_col], hrf=hrf)...)
    BOLDFrame(events, nivolumes, event_time_col, tr_to_event)
end

BOLDFrame

In [60]:
bf = BOLDFrame(nested_by_run[:data], run_niftis[:nifti], event_time_col=:start_secs)
bf.tr_to_event

464×222 Array{Float64,2}:
  0.0           0.0           0.0          …   0.0           0.0       
  0.101658      0.0           0.0              0.0           0.0       
  0.159794      7.93649e-11   0.0              0.0           0.0       
  0.0564006     0.103331      0.0              0.0           0.0       
  0.000158514   0.159103      2.47697e-9       0.0           0.0       
 -0.0152137     0.0556812     0.104996     …   0.0           0.0       
 -0.0127044    -9.58782e-5    0.1584           0.0           0.0       
 -0.00639594   -0.0152424     0.0549664        0.0           0.0       
 -0.0023371    -0.0126534    -0.000347576      0.0           0.0       
 -0.000672901  -0.00635017   -0.0152698        0.0           0.0       
 -0.000160727  -0.00231515   -0.0126022    …   0.0           0.0       
  0.0          -0.000665426  -0.00630458       0.0           0.0       
  0.0          -0.000158721  -0.00229337       0.0           0.0       
  ⋮                                   

# Modeling

## Fixed effects model matrix

This is very straingtforward.  Just generate the individual model matrices, concatenate them, and multiple by the transformer.

Makes me wonder whether it's worth having one big transformation vs. a bunch of little ones (one per pair).  Easy enough to benchmark once I know the general strategy works

In [76]:
function model_matrix(f::Formula, bf::BOLDFrame)

    g = Formula(nothing, f.rhs)

    bf.tr_to_event * vcat((ModelMatrix(ModelFrame(g, events)).m for events in bf.events)...)
end

model_matrix(@formula(_ ~ 0 + category + (1|subject_num)), bf)

464×2 Array{Float64,2}:
  0.0           0.0        
  0.0           0.101658   
  7.93649e-11   0.159794   
  0.103331      0.0564006  
  0.159103      0.000158517
  0.0556812     0.089782   
 -9.58782e-5    0.145696   
 -0.0152424     0.0485705  
 -0.0126534     0.104792   
 -0.00635017    0.249683   
 -0.00231493    0.198105   
  0.10927       0.0465265  
  0.156073      0.0923046  
  ⋮                        
  0.0147794    -0.00704367 
 -0.0254105     0.156515   
 -0.0254667     0.110626   
 -0.0143377     0.0330021  
 -0.0057665     0.157288   
 -0.00180203    0.0981643  
 -0.000461619   0.0144483  
 -8.41892e-5   -0.0143349  
  0.0          -0.0167093  
  0.0          -0.0101895  
  0.0          -0.00431315 
  0.0          -0.00137992 

## Random effects model matrices

MixedModels uses a special `[Scaler|Vector]ReMat` type, which keeps the model matrix and the group indices separately, and has special methods to do linear algebra.  This has the advantage (I think) of using an efficient storage format.  But it will happily fit with a regular sparse/dense matrix, and on the dyestuff data there's very little difference between these three.  The only problem comes from printing the output (and generating the `VarCorr` summary), since the `ReMat` types also keep track of their grouping factor name.

It's not possible to use the normal ReMat types because the HRFs for, say, stimulus types might overlap, so you can't simply say which level each row of the model matrix applies to.  There are two options as I see them:

1. Fit using a "naked" matrix type and don't worry about plugging in the names until after the fact (since the fitting is going to happen internally anyway, with the results copied into some storage format...)
2. Create another `ReMat` subtype that stores the whole matrix and defines/delegates the same methods that MixedModels ReMats do (some linear algebra and size and levels).

I think the second strategy is better long term, but setting up all the methods might be annoying since there are a lot of specialized methods. At least there's a guide to what needs to be covered in the MixedModels.

Internally, a mixed model stores the random effects matrices, fixed effects matrix, and the response in a vector in the `.trms` field.  S

In [97]:
rem = MixedModels.remat(:((1 | run_number)), vcat(bf.events...))

MixedModels.ScalarReMat{Float64,Int64,UInt8}([1,1,1,1,1,1,1,1,1,1  …  4,4,4,4,4,4,4,4,4,4],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0  …  1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0],:run_number,String["(Intercept)"])

In [101]:
size(rem)

(222,4)

In [100]:
rem.f.refs

222-element Array{UInt8,1}:
 0x01
 0x01
 0x01
 0x01
 0x01
 0x01
 0x01
 0x01
 0x01
 0x01
 0x01
 0x01
 0x01
    ⋮
 0x04
 0x04
 0x04
 0x04
 0x04
 0x04
 0x04
 0x04
 0x04
 0x04
 0x04
 0x04